# Seminario 2. Funciones en Python.

Las funciones ofrecen muchas posibilidades en Python, más allá del uso básico que habitualmente hacemos de ellas. Aquí abordamos algunos aspectos poco conocidos y ciertas características que conviene conocer para desarrollar todo el potencial de las funciones en Python.

## Las funciones también son objetos.

En Python las funciones son objetos, al igual que los enteros, las cadenas de caracteres o las listas, de modo que, dada una función `f`, la expresión `type(f)` retornará `<class ‘function'>`.

Como cualquier otro objeto, las funciones poseen atributos que determinan su estado interno. Entre ellos, cabe destacar los siguientes:

* `f.__name__`: nombre de la función (cadena de caracteres)
* `f.__doc__`: descripción de la función (cadena de caracteres)
* `f.__defaults__`: valores por defecto de los parámetros (tupla)
* `f.__globals__`: módulos, variables, etc. que conforman el entorno de ejecución de la función, conocido como *clausura* (diccionario)

Las funciones pueden asignarse a variables y, lógicamente, estas variables podrán ejecutarse (realmente estaremos ejecutando las funciones a las que hagan referencia). Por ejemplo:

    import math
    g = math.sqrt
    g(25)
    
    5.0

Podemos construir listas o diccionarios con nombres de funciones. Por ejemplo:

    import math
    l = [math.log, math.sqrt, math.exp, math.tan]
    for f in l:
        print(f(4))
        
    1.3862943611198906
    2.0
    54.598150033144236
    1.1578212823495775
 
Supongamos que pedimos el nombre de una función. El usuario introducirá una cadena de caracteres. ¿Cómo transformar esa cadena en el nombre (ejecutable) de una función? Muy sencillo: utilizando `eval`.

    import math
    fname = input("Nombre de la función: ")
    f = eval(fname)
    f(4)
    
    Nombre de la función: math.sqrt
    2.0

## Docstrings

El llamado docstring es un comentario multi-línea (encerrado entre triples comillas) que describe la tarea y el modo de uso de la función, y que se incluye justo debajo del prototipo de la función. Internamente, se almacena en el atributo `f.__doc__` (siendo `f` el nombre de la función). Podemos acceder al docstring haciendo `help(f)`. Muchas herramientas de desarrollo, al teclear el nombre de la función y abrir el primer paréntesis, presentan al programador una ventana emergente con el contenido del docstring, lo cual facilita la comprensión de su funcionalidad y el modo de uso (parámetros, valores por defecto, etc.). La definición de cualquier función debería tener la siguiente estructura:

    def f(arg1, arg2, ...):
        """ Descripción de la función:
                precondiciones
                parámetros
                funcionalidad
                valor de retorno
        """
           
        <cuerpo de la función>

## Espacios de nombres

Cada función define un espacio de nombres, es decir, un ámbito propio para variables locales: los parámetros de la función y las variables auxiliares que se crean dentro de la función.

Al ejecutar código en Python, y en particular el código de una función, los símbolos de variables, funciones, etc. se buscan en distintos espacios de nombres. Si un símbolo no se encuentra en el espacio de nombres local (propio de la función), la función lo busca sucesivamente en espacios de nombres superiores en la jerarquía, subiendo en el árbol de llamadas a función hasta llegar al espacio global de nombres (el del programa principal); si tampoco está allí, accede al espacio de nombres del lenguaje Python. Por último, si no encuentra el símbolo, el intérprete de Python genera una excepción de tipo `NameError`.

### Módulos importados
Cada módulo importado crea su propio espacio de nombres, al cual se accede usando el nombre del módulo y el operador punto (`.`). Por ejemplo:

In [6]:
import math

y = math.sqrt(4.0)

Al escribir `math.sqrt`, estamos forzando la búsqueda del símbolo *sqrt* única y exclusivamente en el espacio de nombres del módulo *math*. Las importaciones se pueden hacer de manera que los símbolos del módulo se traigan al espacio de nombres donde aparece la instrucción `import`. Por ejemplo: 

In [7]:
from math import sqrt

y = sqrt(4.0)

Hay que tener cuidado con este tipo de importaciones, ya que podría haber variables o funciones con un mismo nombre en distintos módulos y podríamos estar sobreescribiendo la definición de ciertos símbolos. En general, se recomieda mantener espacios de nombres separados para cada módulo.

### Variables globales (de módulo)
Como ya sabemos, las variables asignadas (creadas) en una función pertenecen al espacio de nombres local. Para forzar su pertenencia al espacio de nombres global, se deben declarar como `global`. Por ejemplo:

    def func(x):
        global nitems
        nitems = nitems + 1
            [...]
En el ejemplo anterior, la variable `nitems` pertenecerá al espacio de nombres global.

### Variables `nonlocal`
También podemos declarar las variables de una función como `nonlocal`, de modo que la variable pertenecerá a un espacio de nombres superior en la jerarquía de llamadas a función (pero no al espacio global). Por ejemplo:

    def procesar(s, c):
        count = 0
        def aux(t):
           nonlocal count
           count = count + 1
           return t[1:]
          [...]

La variable `count` que aparece en la función `aux` se busca en un nivel superior; en este caso, se encuentra en el espacio de nombres local de la función `procesar`.

## Parámetros con valores predeterminados.

Formalmente, los *parámetros* son los nombres que se especifican en la *definición* de la función, mientras que los *argumentos* son las expresiones que se especifican en la *llamada* a la función. Es decir, al llamar a una función se le *pasan* argumentos que son *recibidos* por (copiados en) sus parámetros, que en realidad son referencias a objetos.

Es posible definir funciones con parámetros a los que se les asignan valores predeterminados. De este modo, se convierten en opcionales, salvo que deseemos pasar valores diferentes a los predeterminados. Esta técnica se usa frecuentemente en funciones en las que algunos de sus parámetros se utilizan para alterar algunos aspectos secundarios de su funcionamiento. Por ejemplo, el método `str.find` permite especificar opcionalmente la posición inicial de búsqueda, y adicionalmente la posición final:

In [8]:
s='¡al rico helado! ¡al rico helado!'
print(s.find('rico'))
print(s.find('rico', 5))
print(s.find('rico', 5, 10))

4
21
-1


Este comportamiento se consigue con una definición similar a ésta:

    def find(sub, start=0, end=-1, /):
        [...]

(El significado de `/` en la lista de parámetros se verá [más adelante](#Establecimiento-de-restricciones-en-el-paso-de-argumentos.).)

Los parámetros sin valores predeterminados deben especificarse siempre en primer lugar, ya que en caso contrario daría lugar a confusión. En otras palabras: una vez que se especifica un parámetro con valor predeterminado, todos los que van a continuación también deben tener un valor predeterminado. Por ejemplo, consideremos la siguiente función:

    def f(a=5, b):
        [...]

¿Cómo se interpretaría la llamada `f(3)`? El argumento `3`, por su posición (primero y único), debería ser recibido por el parámetro `a`; sin embargo, como el parámetro `b` no es opcional, ¿no debería recibirlo el parámetro `b`? Para evitar esta situación, Python lo prohíbe sintácticamente:

    def f(a=5, b):
        print(a, b)
	
    SyntaxError: non-default argument follows default argument
    

Es importante tener en cuenta que **los valores predeterminados de los parámetros se evalúan al definir la función**, no al llamarla. Esto puede dar lugar a sorpresas en caso de utilizar objetos mutables. Véamoslo con un ejemplo:

In [9]:
def f(a, l=[]):
    l.append(a)
    return l

print(f(1))
print(f(2))
print(f(3))
print(f('c', ['a','b']))
print(f(4))

[1]
[1, 2]
[1, 2, 3]
['a', 'b', 'c']
[1, 2, 3, 4]


¿Qué ha sucedido? El valor predeterminado de `l` se evaluó al definir la función `f`, y ya no vuelve a evaluarse por muchas veces que la llamemos. Es decir, el objeto `[]` se creó al definir la función, así como la referencia a él desde el parámetro `l`. Entonces, en la primera llamada se añade a esa lista el valor `1`; en la segunda llamada se le añade *a la misma lista* (que sigue referenciada por `l`) el valor `2`, etc. Nótese que esto no se ve afectado porque en otro momento llamemos a `f` pasando una lista distinta al parámetro `l`, ya que en la siguiente llamada en que se use su valor predeterminado, este parámetro vuelve a referenciar el objeto inicial.

## Argumentos *keyword*.

Como alternativa al paso de argumentos posicionales, también es posible llamar a una función especificando los argumentos en la forma `keyword=expresión`, donde `keyword` es el nombre de cualquier parámetro de la función. En este caso, el orden o posición de los argumentos *keyword* es irrelevante.

Es posible combinar argumentos posicionales y argumentos *keyword* en una llamada a una función. Ahora bien, de forma similar a lo que sucede con los parámetros con valores predeterminados, los argumentos posicionales deben especificarse primero, y a continuación deberán ir los argumentos *keyword*. Un ejemplo:

In [10]:
def f(a, b=10, c='hola que tal'):
    # Más adelante veremos una forma más general, simple y elegante de hacer esto
    for arg in 'abc':
        print(f'{arg} = {eval(arg)}')

In [11]:
f(5)

a = 5
b = 10
c = hola que tal


In [12]:
f(b=2, a=1)

a = 1
b = 2
c = hola que tal


In [13]:
f(1, 2, 3)

a = 1
b = 2
c = 3


In [14]:
f(c='caramba', a=100)

a = 100
b = 10
c = caramba


Como se puede ver, mientras se respete la regla de que los argumentos posicionales (si los hay) han de especificarse delante de los argumentos *keyword* (si los hay), podemos realizar las combinaciones que queramos.

Por el contrario, las llamadas a la función `f` que se listan a continuación no son posibles:

    f()                # Se exige al menos un argumento (TypeError)
    f(a=100, 7)        # Argumento posicional tras un argumento keyword (SyntaxError)
    f(100, a=5)        # Dos argumentos para el mismo parámetro (TypeError)
    f(100, d='adiós')  # Argumento keyword inesperado (TypeError)
    

## Empaquetamiento de argumentos en parámetros.

Python proporciona mecanismos para **definir funciones con un número indeterminado de argumentos**. Veamos qué sucede con la siguiente función:

In [15]:
def f(a, b=10, *posargs):
    print(f'a = {a}')
    print(f'b = {b}')
    print(f'posargs = {posargs}')

In [16]:
f(5)

a = 5
b = 10
posargs = ()


In [17]:
f(1, 2, 3, 4, 5, 6)

a = 1
b = 2
posargs = (3, 4, 5, 6)


¿Qué estamos viendo? Al especificar un parámetro precedido de `*`, en ese parámetro se empaquetan todos los *argumentos posicionales* que se especifiquen en la llamada, excepto aquéllos a los que ya les corresponda un parámetro. En este ejemplo, la función `f` tiene dos parámetros específicos (`a` y `b`), y un tercer parámetro `*posargs` que recogerá el resto de argumentos posicionales, empaquetándolos en forma de tupla. Lógicamente, `*posargs` sólo recogerá argumentos posicionales, ya que si en la llamada a `f` utilizamos argumentos *keyword* para `a` o `b`, todos los demás argumentos también habrán de ser *keyword*.

No obstante, y como cabría esperar, también hay una forma de empaquetar argumentos *keyword* en un parámetro:

In [18]:
def f(a, b=10, *posargs, **kwargs):
    print(f'a = {a}')
    print(f'b = {b}')
    print(f'posargs = {posargs}')
    print(f'kwargs  = {kwargs}')

In [19]:
f(5)

a = 5
b = 10
posargs = ()
kwargs  = {}


In [20]:
f(1, 2, 3, 4, 5, 6)

a = 1
b = 2
posargs = (3, 4, 5, 6)
kwargs  = {}


In [21]:
f(1, 2, 3, 4, c='hola', d='adiós')

a = 1
b = 2
posargs = (3, 4)
kwargs  = {'c': 'hola', 'd': 'adiós'}


In [22]:
f(1, c='hola', b=2, d='adiós')

a = 1
b = 2
posargs = ()
kwargs  = {'c': 'hola', 'd': 'adiós'}


Al especificar un parámetro precedido de `**` (`**kwargs`), en éste se empaquetan en forma de diccionario todos los argumentos *keyword* que haya en la llamada, excepto aquéllos para los que exista un parámetro específico. En dicho diccionario cada clave es el nombre del argumento (en forma de cadena de caracteres), y cada valor –lógicamente– su valor. Obsérvese en el último ejemplo de llamada a `f` cómo el argumento `b=2` no ha sido recogido en `**kwargs`, puesto que ya existe un parámetro denominado `b` en la definición de la función.

En caso de utilizar ambos tipos de parámetros, tanto `*` como `**`, hay que especificar primero los argumentos posicionales y después los argumentos *keyword*.

Un caso extremo sería una función que sólo contara con dos parámetros `*` y `**`. En ese caso, el primero recogería todos los argumentos posicionales (si los hubiera), y el segundo todos los argumentos *keyword* (si los hubiera) especificados en cualquier llamada a esa función. Véamoslo con un ejemplo:

In [23]:
def g(*posargs, **kwargs):
    for arg in posargs:
        print(arg)
    for k, v in kwargs.items():
        print(f'{k} = {v}')

In [24]:
g(1,2,3, a='uno', b='dos', c='tres')

1
2
3
a = uno
b = dos
c = tres


In [25]:
g(1,2,3)

1
2
3


In [26]:
g(a='uno', b='dos', c='tres')

a = uno
b = dos
c = tres


## Desempaquetamiento y empaquetamiento en Python.

En este punto, conviene explicar que Python posee un operador (`*`) que permite desempaquetar *iterables*. En otro seminario hablaremos sobre iterables; de momento, supongamos que se trata de simples secuencias. En realidad, se usa de forma implícita en expresiones como la que sigue (que ya hemos utilizado en varias ocasiones):

    a, b, c = 1, 2, 3

Para crear tuplas no son necesarios los paréntesis (si no hay ambigüedad sintáctica); basta con las comas. Por esa razón, estas cuatro expresiones son equivalentes:

    a, b, c = 1, 2, 3
    a, b, c = (1, 2, 3)
    (a, b, c) = 1, 2, 3
    (a, b, c) = (1, 2, 3)

El mero hecho de que a la izquierda de una asignación figure una tupla (o una lista) de variables hace que Python automáticamente realice un *desempaquetamiento* de la secuencia situada a la derecha. Por supuesto, la expresión de la derecha no tiene por qué ser un literal; cualquier secuencia (en realidad, cualquier objeto iterable) es válido. La única restricción es que *el número de elementos a ambos lados de la asignación sea el mismo*.

Otro caso en el que el desempaquetamiento se produce de manera automática lo encontramos al recorrer mediante un `for` una lista de tuplas. Por ejemplo:

In [27]:
lt = [(1, 'a', 3.5), (2, 'b', 0.03), (3, 'c', 5.67), (4, 'd', -2.98)]
for n, c, x in lt:
    print(f'{n} {c} {x:6.2f}')

1 a   3.50
2 b   0.03
3 c   5.67
4 d  -2.98


Si bien en asignaciones como las que hemos visto el desempaquetamiento tiene lugar de forma automática e implícita, también es posible desempaquetar secuencias en otras expresiones, en cuyo caso sí hemos de utilizar el operador `*`:

In [28]:
t1 = (1, 2, 3)
t2 = (4, 5)
l = [*t1, 'hola', *t2]
print(l)

[1, 2, 3, 'hola', 4, 5]


También es posible desempaquetar diccionarios, utilizando el operador `**`. El funcionamiento es muy similar. Veamos un ejemplo sencillo:

In [29]:
d1 = {'hola': 4, 'qué': 3, 'tal': 3}
d2 = {'adiós': 5, 'muy':3 , 'buenas': 6}
d = {**d1, 'caramba': 7, **d2}
print(d)

{'hola': 4, 'qué': 3, 'tal': 3, 'caramba': 7, 'adiós': 5, 'muy': 3, 'buenas': 6}


### Empaquetamiento.

Python permite utilizar el operador `*` en la parte izquierda de una asignación, siempre que ésta consista en una lista de variables. Sólo se puede utilizar con una de las variables de la lista (las demás se dice que son *obligatorias*). En este caso, su efecto es *empaquetar* los elementos “sobrantes” del desempaquetamiento de la secuencia a la derecha de la asignación en la variable a la que se le ha aplicado el `*`. Lo mejor es verlo con un ejemplo:

In [30]:
a, b, *c = range(10)  # range(10) es un iterable
print(a, b, c, sep=' || ')

a, *b, c = 'CADENA'
print(a, b, c, sep=' || ')

*a, b, c = 'hola', 'adiós'  # ¡esto es una tupla!
print(a, b, c, sep=' || ')

0 || 1 || [2, 3, 4, 5, 6, 7, 8, 9]
C || ['A', 'D', 'E', 'N'] || A
[] || hola || adiós


Como se puede apreciar, el empaquetamiento siempre resulta en una lista. Las variables obligatorias (las que no llevan el `*`) siempre deben recibir un elemento, por lo que en la secuencia de la derecha debe haber al menos tantos elementos como variables obligatorias a la izquierda. El resto de los elementos, si quedan, se empaquetan en la variable con `*`.

Este mecanismo es similar al que actúa cuando una función posee un parámetro `*` en el que se empaquetan los argumentos posicionales “sobrantes” de la llamada.

### Desempaquetamiento de argumentos de funciones.

Obviamente, también podemos usar los operadores de *desempaquetamiento* de secuencias (`*`) y diccionarios (`**`) en los argumentos de una llamada a una función. Veamos algunos ejemplos:

In [31]:
def f(a, b, c, d):
    for arg in 'abcd':
        print(f'{arg} = {eval(arg)}') # arg es el nombre de uno de los parámetros

In [32]:
l = [1, 2, 3, 4]
f(*l)

a = 1
b = 2
c = 3
d = 4


Aquí simplemente se ha desempaquetado la lista `l` (podría tratarse de cualquier tipo de secuencia o iterable) en la llamada a `f`, logrando el mismo efecto que con `f(1, 2, 3, 4)`.

Desempaquetando un diccionario, logramos el mismo efecto que si utilizásemos argumentos *keyword*:

In [33]:
dicc = {'c': 10, 'd': 20}
f(1, 5, **dicc)

a = 1
b = 5
c = 10
d = 20


Aquí podemos ver, además, que podemos usar el desempaquetamiento sólo para parte de los argumentos. Es más, podemos combinar ambos tipos de desempaquetamiento:

In [34]:
l = [1, 5]
dicc = {'c': 10, 'd': 20}
f(*l, **dicc)

a = 1
b = 5
c = 10
d = 20


Con toda lógica, la sintaxis del desempaquetamiento de argumentos, empleando los operadores `*` y `**`, es totalmente coherente con la sintaxis del [empaquetamiento de argumentos en parámetros](#Empaquetamiento-de-argumentos-en-parámetros.).

Obviamente, no hay inconveniente alguno en emplear ambas técnicas: podemos definir funciones con parámetros que empaquetan argumentos posicionales y/o argumentos *keyword*, y usar el desempaquetamiento de argumentos a la hora de llamarlas. Recuperemos una función que ya utilizamos como ejemplo más arriba:

In [35]:
def g(*posargs, **kwargs):
    for arg in posargs:
        print(arg)
    for k, v in kwargs.items():
        print(f'{k} = {v}')

In [36]:
l = [10, 20, 30]
d = {'r': 'hola', 's': 'qué', 't': 'tal'}

g(*l, **d)

10
20
30
r = hola
s = qué
t = tal


In [37]:
g(5, *l, **d, cosa=1000)

5
10
20
30
r = hola
s = qué
t = tal
cosa = 1000


Como se puede ver, podemos combinar como queramos argumentos individuales y desempaquetados, tanto posicionales como *keyword*, siempre que respetemos la restricción de que los argumentos posicionales deben ir por delante de los argumentos *keyword*.

## Establecimiento de restricciones en el paso de argumentos.

Al definir una función, nos puede interesar que a ciertos parámetros se les puedan pasar sólo argumentos posicionales, sólo argumentos *keyword* o de ambos tipos. Esto se puede lograr mediante los parámetros especiales `/` y `*`:

    def f(pos1, pos2, /, pos_o_kwd, *, kwd1, kwd2):
          -----------    ----------    ----------
            |              |                |
            |   Posicional o keyword        |
            |                                -- Sólo keyword
             -- Sólo posicionales

El uso de los símbolos `/` y `*` es opcional. Se puede utilizar sólo uno de ellos o ambos. En este último caso, el símbolo `/` debe ir siempre antes que el símbolo `*`.

Cuando al comienzo de este seminario tratábamos de los [parámetros con valores predeterminados](#Parámetros-con-valores-predeterminados.), pusimos como ejemplo el método `str.find`, y decíamos que su definición de parámetros es similar a ésta:

    def find(sub, start=0, end=-1, /):

Lo que se pretende con esto es impedir que se puedan pasar argumentos *keyword* a este método, y podemos ver que así sucede:

    'Hola y adiós'.find('y', end=8)
                [...]
                
    TypeError: find() takes no keyword arguments

Otro ejemplo del uso de estas restricciones lo podemos ver en el método `list.sort`, cuya definición es más o  menos así:

    def sort(/, *, key=None, reverse=False):

Los argumentos de esta función (todos opcionales) han de ser obligatoriamente de tipo *keyword*:

    lt = [(1, 5), (2, 0), (5, 2), (7, 1)]
    lt.sort(lambda t: t[1])
             [...]
             
    TypeError: sort() takes no positional arguments

## Expresiones Lambda.

Las expresiones `lambda` se usan para definir pequeñas funciones anónimas, con la siguiente sintaxis:

    lambda arg1, arg2, ... : expresión(arg1, arg2, ...)

Una función definida de esta forma retornará el resultado de evaluar la expresión de la derecha al sustituir arg1, arg2, ... por los valores que hayamos pasado en la llamada. Por ejemplo:

    maximo = lambda x, y: x if x>y else y
    maximo(5, -4)
    
    5

Otro ejemplo:

    borrar = lambda s, sub: ''.join(s.split(sub))
    borrar("canción de cuna para una marmota", "una")
    
    'canción de c para  marmota'

A primera vista puede resultar difícil encontrar utilidad a las expresiones `lambda`, pero en ciertos contextos resulta extremadamente conveniente poder definir una función en el mismo lugar donde se utiliza, sin necesidad de definirla aparte de la manera convencional. Precisamente, un poco más arriba se ha presentado el método `list.sort` como ejemplo de contexto en el que se obliga a utilizar argumentos *keyword*, haciendo uso de una expresión `lambda`. Vamos a recuperar ese ejemplo, si bien (por simplicidad) emplearemos la función `sorted`, en lugar del método `list.sort`.

La función `sorted` (al igual que el método `list.sort`) posee un parámetro `key`, a través del cual se suministra una función que actúa como *clave de ordenación*. Antes de comparar dos elementos de la lista (para saber cuál va antes), `sorted` les aplica el método especificado a través del parámetro `key`. En ausencia del argumento `key`, la función `sorted` utiliza como claves los propios elementos de la secuencia. Cuando deseamos usar como clave de ordenación una combinación de ciertas partes de cada elemento, el parámetro `key` permite precisamente definir dicha combinación. Considérese, por ejemplo, la siguiente lista de tuplas:

In [33]:
lt = [(1, 5), (2, 0), (5, 2), (7, 1)]

Supongamos que queremos ordenarla pero *utilizando como clave el segundo elemento de las tuplas*. Normalmente, `sort` ordenaría la lista comparando los valores de las tuplas de izquierda a derecha, empezando por el primero, después el segundo, etc.:

In [34]:
print(sorted(lt))

[(1, 5), (2, 0), (5, 2), (7, 1)]


¿Cómo podemos usar el segundo elemento de las tuplas como clave? La solución es muy simple: pasándole en el parámetro `key` una función anónima que retorne el 2º elemento de su argumento. Esto lo podemos conseguir mediante una expresión `lambda` del siguiente modo:

In [35]:
print(sorted(lt, key=lambda t: t[1]))

[(2, 0), (7, 1), (5, 2), (1, 5)]


¿Y si quisiéramos ordenar por la suma de todos los elementos de cada tupla?

In [36]:
print(sorted(lt, key=lambda t: sum(t)))

[(2, 0), (1, 5), (5, 2), (7, 1)]


Veamos un último ejemplo de aplicación de las expresiones `lambda`. Esta vez definimos un diccionario con sendas funciones de conversión de grados Fahrenheit a Kelvin y de grados Celsius a Kelvin:

In [38]:
t = { "F2K": lambda g: 273.15 + (g-32) * 5/9, "C2K": lambda g: 273.15 + g }
print(t["F2K"](32))
print(t["C2K"](17))

273.15
290.15


## Las funciones `map()`, `filter()` y `reduce()`.

Las funciones `map()` y `filter()` forman parte del lenguaje Python, mientras que `reduce()` viene incluida en el módulo `functools`. Estos métodos se han integrado en Python pero provienen de los así llamados *lenguajes funcionales*, basados en un paradigma de programación basado exclusivamente en funciones y métodos, que se combinan entre sí, generando, simplificando, transformando o reduciendo secuencias de datos. Pese a ser herramientas muy extendidas, existen alternativas más eficientes en Python, como las *comprehensions*, que veremos en un seminario posterior.

La expresión `map(func, items1, items2, ..., itemsN)` genera un iterable con la salida de la función `func`, cuyos N argumentos de entrada se toman de las N secuencias (listas, iterables, ...) que le siguen en la lista de argumentos. El tamaño del iterable de salida es el de las más corta de las secuencias de entrada. Por ejemplo:

In [38]:
def centro(*p):
    x = 0.0
    y = 0.0
    for i in range(len(p)):
        x += p[i][0]
        y += p[i][1]
    return x/len(p), y/len(p)

lst1 = [(2, 3), (5, 6), (8, 9)]
lst2 = [(-3, 5), (-1, 4)]
lst3 = [(1, 1), (2, 2), (3, 3), (4, 4)]
g = map(centro, lst1, lst2, lst3)
for c in g:
    print(c)

(0.0, 3.0)
(2.0, 4.0)


La expresión `filter(func, t)` genera un iterable con los elementos del iterable `t` que cumplen la condición expresada por la función `func` (que actúa como filtro). La función `func()` toma un único valor como argumento y retorna `True` o `False`. Por ejemplo:

In [39]:
import math

def centro(*p):
    x = 0.0
    y = 0.0
    for i in range(len(p)):
        x += p[i][0]
        y += p[i][1]
    return x/len(p), y/len(p)

lst1 = [(2, 3), (5, 6), (8, 9)]
lst2 = [(-3, 5), (-1, 4)]
lst3 = [(1, 1), (2, 2), (3, 3), (4, 4)]
g = map(centro, lst1, lst2, lst3)
f = filter(lambda p: math.sqrt(p[0]*p[0]+p[1]*p[1])>4, g)
for c in f:
    print(c)

(2.0, 4.0)


La salida de la función `filter`es un iterable formado por las tuplas del iterable g (generado por una función `map`) que verifican la condición del filtro. Como vemos, la función filtro se ha definido a través de una expresión `lambda`.

La expresión `functools.reduce(func, t, [initial_value])` repite de manera iterativa la operación `func(a, b)` sobre los elementos del iterable `t`, de acuerdo al siguiente esquema:

1. toma los dos primeros elementos del iterable, `a` y `b`, llama a `func(a, b)` obteniendo un resultado `r1`;
2. toma el siguiente elemento `c` del iterable, llama a `func(r1, c)`, obteniendo `r2`;
3. toma el siguiente elemento `d` del iterable, llama a `func(r2, d)`, obteniendo `r3` ...
4. ... y así sucesivamente hasta terminar con el iterable, retornando el último resultado calculado.

Si se suministra un valor inicial, este se usa como punto de partida, calculando en primer lugar `func(initial_value, a)`. Si el iterable no produce ningún valor, `functools.reduce()` retorna el valor inicial.
Si no se suministra un valor inicial y el iterable `t` no produce ningún valor, se produce una excepción de tipo `TypeError`. Veamos unos ejemplos:

In [39]:
import functools
import math

def modulo(p):
    return math.sqrt(p[0]*p[0]+p[1]*p[1])

l = [(2, 3), (5, 6), (8, 9), (3, 6), (1, 2)]
r = functools.reduce(lambda p, q: p if modulo(p)>modulo(q) else q, l, (0,0))
print(r)

(8, 9)


In [40]:
# Iterable vacío con valor inicial
r = functools.reduce(lambda p, q: p if modulo(p)>modulo(q) else q, [], (0,0))
print(r)

(0, 0)


In [41]:
# Iterable vacío sin valor inicial
r = functools.reduce(lambda p, q: p if modulo(p)>modulo(q) else q, [])

TypeError: reduce() of empty sequence with no initial value

Combinando las funciones `map()`, `filter()` y `reduce()`, y usando expresiones `lambda` para definir funciones anónimas donde convenga, se pueden atacar operaciones de procesamiento de datos muy complejas. Sin embargo, como se ha dicho, las implementaciones en Python de estas funciones no son especialmente eficientes, por lo que no se recomienda utilizarlas con asiduidad. 